## VCF parsing program
---
* 프로그램이 위치한 경로에 `vcf` 폴더와 `gff3` 폴더를 생성해야 한다
* 각각의 폴더 안에는 `vcf` 파일과 `gff3` 파일을 하나씩 위치시켜야 하며, 각각의 파일명을 입력으로 받는다
* 각 입력 파일은 염색체 하나의 정보만을 가지고 있어야 한다
* vcf 파일에 있는 각 variant가 어느 유전자에 해당하는 것인지 프로그램과 같은 경로에 `result.csv` 파일로 출력한다.
* 요구사양: python 3 or greater (pandas, numpy module requested)
---

In [1]:
print('VCF parsing program by ---------------')
print('''
  |_  ||_   _| | |  | | / _ \ | \ | |
    | |  | |   | |  | |/ /_\ \|  \| |
    | |  | |   | |/\| ||  _  || . ` |
/\__/ / _| |_  \  /\  /| | | || |\  |
\____/  \___/   \/  \/ \_| |_/\_| \_/
''')
print('-------------------------------------')

VCF parsing program by ---------------

  |_  ||_   _| | |  | | / _ \ | \ | |
    | |  | |   | |  | |/ /_\ \|  \| |
    | |  | |   | |/\| ||  _  || . ` |
/\__/ / _| |_  \  /\  /| | | || |\  |
\____/  \___/   \/  \/ \_| |_/\_| \_/

-------------------------------------


In [2]:
import pandas as pd
import numpy as np

In [3]:
#vcf ex: homo_sapiens-chrY
#gff3 ex: Homo_sapiens.GRCh38.96.chromosome.Y

vcf_name = input('vcf 파일의 이름을 적어주세요 (확장자 제외): ')
gff3_name = input('gff3 파일의 이름을 적어주세요 (확장자 제외): ')

import1 = './vcf/'+vcf_name+'.vcf'
import2 = './gff3/'+gff3_name+'.gff3'

In [4]:
#gff3 file read
gff3 = pd.read_csv(import2, sep='\t', comment='#', error_bad_lines=False)

gff3_gene = gff3.iloc[:,[3,4,6,-1]]
gff3_gene.columns.values[[0,1,2,3]] = ['start', 'end', '..1', 'target']

#drop gff3 except + strand
m = gff3_gene['..1'] == '+'
gff3_gene = gff3_gene[m]

#중복 제거
gff3_gene.drop_duplicates(['start']) 

#index 초기화
gff3_gene = gff3_gene.iloc[:,[0,1,-1]].reset_index(drop = True)

#drop gff3 except gene row
for j in range(gff3_gene.shape[0]):
    if('Name=' not in gff3_gene['target'][j]):
        gff3_gene = gff3_gene.drop(j, 0)

#target 형식 변경        
def nameprocess(row):
    return row['target'].split('Name=')[1].split(';')[0].split('-')[0]
         
gff3_gene['target'] = gff3_gene.apply(nameprocess, axis = 1)
    
#index 초기화
gff3_gene = gff3_gene.reset_index(drop = True)

gff_start = gff3_gene['start']
gff_end = gff3_gene['end']

In [5]:
#vcf file read
vcf1 = pd.read_csv(import1, sep='\t', header=38, error_bad_lines=False)

vcf_mod = vcf1.iloc[:,[1,3,4]]

#position check
m = vcf_mod["POS"] >= min(gff_start)
m2 = vcf_mod["POS"] <= max(gff_end)
vcf_mod = vcf_mod[m][m2]

vcf_mod = vcf_mod.reset_index(drop = True)

/biotools/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [6]:
#dataframe to numpy matrix
gff_np = gff3_gene.as_matrix()
vcf_np = vcf_mod.as_matrix()

/biotools/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/biotools/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print('파일 불러오기 완료. 데이터 처리 중...')

#processing

pos = vcf_np[:,0]
ref = vcf_np[:,1]
alt = vcf_np[:,2]

gff_start = gff_np[:,0]
gff_end = gff_np[:,1]
target = gff_np[:,2]

temp = []

for i in range(vcf_np.shape[0]):
    pos_t = pos[i]
    for j in range(gff_np.shape[0]):
        if(pos_t in range(gff_start[j], gff_end[j]+1)): #start~end 사이에 있는지 check
            temp.append([pos_t, ref[i], alt[i], target[j]])
                      
result = pd.DataFrame(temp, columns = ['POS','REF','ALT','GENE'])
result.to_csv('./result.csv', header=True)

In [7]:
print('-------------------------------------')
print('''
|  _  \|  _  || \ | ||  ___|| |
| | | || | | ||  \| || |__  | |
| | | || | | || . ` ||  __| | |
| |/ / \ \_/ /| |\  || |___ |_|
|___/   \___/ \_| \_/\____/ (_)
''')
print('file export: result.csv -------------')

-------------------------------------

|  _  \|  _  || \ | ||  ___|| |
| | | || | | ||  \| || |__  | |
| | | || | | || . ` ||  __| | |
| |/ / \ \_/ /| |\  || |___ |_|
|___/   \___/ \_| \_/\____/ (_)

file export: result.csv -------------
